# Authentication

## Import
There is only one thing we really need, and that is the [TrendMinerClient](../classes/client.rst#TrendMinerClient) class. Everything we can do with `trendminer_interface` is implemented on that class.

In [1]:
from trendminer_interface import TrendMinerClient

## Authenticating
As a first step, we will always need to create a [TrendMinerClient](../classes/client.rst#TrendMinerClient) instance which is authenticated to send and receive requests to our appliance. The [TrendMinerClient](../classes/client.rst#TrendMinerClient)  instance will always attempt to refresh authentication once expired, though through inactivity or prolonged use (couple of hours), the authentication will eventually expire and you will have to log in again. The [TrendMinerClient](../classes/client.rst#TrendMinerClient)  instance does not keep any sensitive information like passwords in memory.

To gain access via API (as we are doing in `trendminer_interface`) we will always need a **client ID** and **client Secret**, which [need to be generated in ConfigHub](https://documentation.trendminer.com/en/clients.html) by an admin.

We typically want to set a **timezone** for our client. All returned timestamps are then given in this particular timezone. The timezone defaults to UTC.

To make the example clear, I am simply inputting (fake) passwords as strings. However, **you should never put your examples in plain text**. In [this blog post](https://martinheinz.dev/blog/59), under 'storing securely', you can find some better practices. In further code examples, I will use the [keyring](https://pypi.org/project/keyring/) package to retrieve passwords securely from my keychain. Note that that to use this package in your own code, you will first have to install this package separately and then make sure your passwords are saved.

### Client user
A client user is only accessible via APIs, so it is not possible to log into TrendMiner via the UI as a client user. A client user is intended only to retrieve data. It is not recommended to use a client user to save work organizer objects, as (i) they will never be accessible via the UI, and (ii) unexpected behaviour might occur as this behaviour is unintended and not thoroughly tested.

Just like a regular user, a client user needs to be given access to specific datasources in ConfigHub in order to access them.

Simply a **client ID** a **client secret** suffice to authenticate to the server as a client. 

In [2]:
url = 'https://cs.trendminer.net/'
client_id = 'wdanielsclient'
client_secret = '282fYzZjTGjxWzJ8viwHswk6Fu9ovop5z'

In [3]:
# The given client secret is simply for illustration; let's get our actual client_secret more securely
import keyring
client_secret = keyring.get_password(url, client_id)

In [4]:
client = TrendMinerClient(
    url=url,
    client_id=client_id,
    client_secret=client_secret,
    tz='Europe/Brussels', 
)

/Users/wouter.daniels/PycharmProjects/tm-python-interface/trendminer_interface/client.py:183: VersionMismatchWarning: This SDK version was tested for use with TrendMiner versions [2024.R1.0 | 2024.R1.1 | 2024.R2.0 | 2024.R2.1] while your TrendMiner version is [2024.R3.1-04]. Some functionality might not work as expected. 
  warnings.warn(


### User client
We can authenticate as a user by providing a name and password in addition to client ID and secret. Authenticating as a user is required if we want access or create that user's saved work organizer items.

We are still required to provide a valide client id and secret to authenticate as a user. It will be the user's access rights that determine the datasources we have access to, regardless of the access rights assigned to the provided client id.

Note that **authenticating as a user only works for local users**. Providing your LDAP or SAML user details does not work (remember, TrendMiner does not and should not know these user credentials). This means that for some use cases, a dedicated local user will need to be created.

In [5]:
username ='wdaniels'
password = 'MyPassw0rd!'

In [6]:
# The given password is simply for illustration; let's get our actual password more securely
password = keyring.get_password(url, username)

In [7]:
client = TrendMinerClient(
    url=url,
    client_id=client_id,
    client_secret=client_secret,
    username=username,
    password=password,
    tz='Europe/Brussels',
)

## Doing things
Once we have our authenticate [TrendMinerClient](../classes/client.rst#TrendMinerClient) instance, we can start interacting with the appliance. For example, let's search for some tags. For a full overview of what you can do with `trendminer_interface`, please refer to the rest of the documentation.

In [10]:
client.tag.search(name='TM4-BP2*.1')

[<< Tag | TM4-BP2-LEVEL.1 >>,
 << Tag | TM4-BP2-PRODUCT.1 >>,
 << Tag | TM4-BP2-TEMP.1 >>,
 << Tag | TM4-BP2-QUAL.1 >>,
 << Tag | TM4-BP2-CW.1 >>,
 << Tag | TM4-BP2-PRESSURE.1 >>,
 << Tag | TM4-BP2-CONC.1 >>,
 << Tag | TM4-BP2-UTIL.1 >>,
 << Tag | TM4-BP2-SPEED.1 >>]

## Troubleshooting and customization
- Adding `verify=False` to TrendMinerClient disables the checking the SSL certificates on requests, **avoiding SSL verification errors** that occur when the certificates of your TrendMiner appliance are expired or self-signed.
- When you get an error claiming the **account is not fully set up**, that either means the password has not yet been changed by the user after it has been (re)set by the admin (do this by logging into TrendMiner manually), or that you are trying to log in as and LDAP/SAML user (which is not possible).
- You can assign the [proxies](https://requests.readthedocs.io/en/latest/user/advanced/#proxies) parameter in TrendMinerClient.
- You can assign your own [timeout](https://requests.readthedocs.io/en/latest/user/advanced/#timeouts) parameter in TrendMinerClient.